In [1]:
#Importamos las modulos
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import time
from numpy import linalg as LA
import random
from time import time
%matplotlib qt

In [2]:
#Definimos la funcion a minimizar
def f(x):
    fx = 100*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 + 90*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 -(20*x[0]+40*x[1])
    return fx

In [3]:
#Definir funcion para calcular la matriz del gradiente
def gradient(x,delta):
    grad=np.zeros(2)
    grad[0]=(f([x[0]+delta,x[1]])- f([x[0]-delta,x[1]]))/(2*delta)
    grad[1]=(f([x[0],x[1]+delta])- f([x[0],x[1]-delta]))/(2*delta)
    return grad

In [4]:
#Utilizamos el algoritmo de golden section para encontrar alpha optimo
def golden(x,search,xi,eps):
    a = xi[0];
    b = xi[1];
    tau = 0.381967;
    alpha1 = a*(1-tau) + b*tau;
    alpha2 = a*tau + b*(1-tau);
    falpha1 = f(x+alpha1*search);
    falpha2 = f(x+alpha2*search);
    for i in range(100):
        if falpha1 > falpha2:
            a = alpha1;
            alpha1 = alpha2;
            falpha1 = falpha2;
            alpha2 = tau*a + (1-tau)*b;
            falpha2 = f(x+alpha2*search);
        else:
            b = alpha2;
            alpha2 = alpha1;
            falpha2 = falpha1;
            alpha1 = tau*b + (1-tau)*a;
            falpha1 = f(x+alpha1*search);
        if np.abs(f(x+alpha1*search)- f(x+alpha2*search)) < eps :
            break;
    return alpha1,falpha1

In [5]:
#Graficamos la funcion de prueba
X1 = np.arange(-5, 5, 0.01)
X2 = np.arange(-5, 5, 0.01)
x1, x2 = np.meshgrid(X1, X2)
z = f([x1,x2])
fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
surf = ax.plot_surface(x1, x2, z, cmap=cm.autumn, linewidth=0, antialiased = False, alpha=0.2)
plt.show()

In [6]:
#Variable en comun de los métodos
delta = 1e-3 
ep1 = 1e-3
puntosBFGS = []

In [7]:
def graficaUnaaUna(puntos, nombreMetodo):
    #Generamos valores ramdom para generar un color diferente de grafica
    r = random.random()
    b = random.random()
    g = random.random()
    color = (r, g, b)
    aux = 0
    x = np.linspace(-1.3, 1.3, 100)
    y = np.linspace(-3, 1.3, 100)
    xx, yy = np.meshgrid(x, y)
    z = f([xx, yy])
    plt.contour(x,y,z,20)
    plt.plot(-1,1, 'ro--',  markersize=6)
    plt.plot(0.5,0, 'go--', markersize=6)
    #Graficamos los trazos (direcciones) que va generando el algoritmo
    for pts in puntos:
        if(aux==len(puntos)-1):
            plt.plot(pts[0],pts[1], c=color, linewidth=2, label=str(nombreMetodo))
        else:
            plt.plot(pts[0],pts[1], c=color, linewidth=2)
        aux = aux + 1
    plt.legend()    
    plt.show

## BFGS method

In [8]:
start_time = time() #Conteo para tiempo
iteraciones = 1 #Conteo de numero de iteraciones
#Test the point Xi+1 for optimality. If ||∇fi+1|| ≤ 𝜀, where 𝜀 is a small quantity,
#take X* ≈ X i+1 and stop the process. Otherwise, go to step 5.
#1. Start with an initial point X1
xi = [-1,1]
xi_1 = xi
#n × n positive definite symmetric matrix [B1] as an initial estimate of the inverse of the Hessian matrix of f
#In the absence of additional information, [B1] is taken as the identity matrix [I].
Bi = np.eye(len(xi))
print(f(xi))
fx_prev=f(xi)
gF_xi = gradient(xi_1, delta)
#2. Compute the gradient of the function, ∇fi, at point Xi, and set
Si = np.matmul(Bi,gF_xi)
#3. Find the optimal step length λ∗i in the direction Si and set
alpha, fx = golden(xi_1,Si,xi,ep1)
xi_1 = xi - alpha*Si
for i in range(1000): #Proceso iterativo del algoritmo
    #Compute the gradient vector ∇f1 = ∇f(X1) and set the iteration number as i = 1.
    gF_xi = gradient(xi, delta)
    #2. Compute the gradient of the function, ∇fi, at point Xi, and set
    Si = np.matmul(np.dot(1, Bi),gF_xi)
    #3. Find the optimal step length λ∗i in the direction Si and set
    alpha, fx = golden(xi_1,Si,xi,ep1)
    xi_1 = xi + np.dot(alpha, Si)
    #4. Test the point Xi+1 for optimality. If ||∇fi+1|| ≤ 𝜀, where 𝜀 is a small quantity, take X* ≈ X i+1 and stop the process. Otherwise, go to step 5.
    if(LA.norm(gF_xi)<ep1 or abs(fx-fx_prev)<ep1):
        print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, xi[0], xi[1],fx,LA.norm(gF_xi)))
        break
    else:
        g1 = np.reshape(gradient(xi_1, delta),(2,1)) - np.reshape(gF_xi,(2,1))
        d1 = np.reshape(np.array(xi),(2,1)) - np.reshape(xi_1,(2,1))
        d1d1t = np.dot(d1, np.reshape(d1,(1,2)))
        d1tg1 = np.matmul(d1.transpose(),g1)
        d1g1t = np.matmul(d1, g1.transpose())
        g1d1t = np.matmul(g1, d1.transpose())
        g1tb1g1 = np.matmul(np.matmul(g1.transpose(), Bi), g1)
        d1g1tb1 = np.matmul(d1g1t,Bi)
        b1g1d1t = np.matmul(Bi, g1d1t)
        Bi = Bi + ((1+(g1tb1g1/d1tg1))*(d1d1t/d1tg1)) - (d1g1tb1/d1tg1) - (b1g1d1t/d1tg1) 
    puntosBFGS.append([[xi_1[0],xi[0]],[xi_1[1],xi[1]]])
    xi = xi_1
    fx_prev = fx
    iteraciones += 1
    print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, xi[0], xi[1],fx,LA.norm(gF_xi)))
elapsed_time = (time() - start_time)*1000
print("Tiempo transcurrido: %f" % elapsed_time)
print("Iteraciones: %d" % iteraciones)
#Graficamos el resultado del Steepest descent method
graficaUnaaUna(puntosBFGS, 'BFGS method')    

270.29416855008
0	[1.2420,-2.7043]		55.8889		444.3158
1	[0.1960,-1.9523]		74.3165		443.6934
2	[1.1232,-1.2909]		34.0478		37.0109
3	[1.2709,-0.9555]		26.8304		67.5989
4	[0.9382,-0.1791]		-0.0429		90.7990
5	[0.8071,-0.2531]		-4.1299		55.0008
6	[0.6996,-0.1247]		-6.2440		16.2680
7	[0.5770,-0.0139]		-7.1258		9.5776
8	[0.5498,-0.0090]		-7.2601		9.9599
9	[0.5271,-0.0059]		-7.3280		6.2747
10	[0.5150,-0.0034]		-7.3452		3.0428
11	[0.5094,-0.0023]		-7.3499		1.6811
12	[0.5058,-0.0015]		-7.3518		1.0342
13	[0.5058,-0.0015]		-7.3525		0.6400
Tiempo transcurrido: 6.977797
Iteraciones: 14
